In [37]:
# !pip3 install wordcloud

In [1]:
import nltk
from nltk.corpus import PlaintextCorpusReader
import re
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from nltk.tokenize import sent_tokenize

import gensim
from gensim import corpora
from gensim import similarities
from gensim import models

from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
import seaborn as sns

## EDA

### EDA at words level

In [46]:
file_directory = '/Users/wyf/Downloads/is450/text-mining-project/text'

filename_pattern = '.+\.txt'

corpus = PlaintextCorpusReader(file_directory, filename_pattern, encoding='latin-1')
fids = corpus.fileids()
print(fids)
print("Number of documents in the collection: ", len(fids))

docs_len = [len(corpus.words(f)) for f in fids]
print("Number of words in each of the books: ", docs_len)

['10002.txt', '10007.txt', '10852.txt', '11.txt', '11870.txt', '120.txt', '12406.txt', '13117.txt', '13610.txt', '13635.txt', '13791.txt', '13888.txt', '13944.txt', '14328.txt', '146.txt', '15478.txt', '15479.txt', '15480.txt', '15489.txt', '16.txt', '16116.txt', '16331.txt', '1656.txt', '16702.txt', '16965.txt', '1727.txt', '17388.txt', '17851.txt', '18183.txt', '18843.txt', '18880.txt', '1951.txt', '19594.txt', '1mart10.txt', '20203.txt', '20467.txt', '20521.txt', '20778.txt', '2130.txt', '22460.txt', '23609.txt', '23680.txt', '24506.txt', '24518.txt', '24708.txt', '2488.txt', '2529.txt', '26377.txt', '26867.txt', '2892.txt', '2rbnh10.txt', '3029.txt', '31529.txt', '31547.txt', '31671.txt', '32032.txt', '345.txt', '34901.txt', '35534.txt', '375.txt', '3800.txt', '38750.txt', '389.txt', '41479.txt', '4339.txt', '4352.txt', '4363.txt', '51233.txt', '514.txt', '5200.txt', '529.txt', '55.txt', '599.txt', '624.txt', '636.txt', '6763.txt', '73.txt', '7370.txt', '7524.txt', '7crmp10.txt', '

In [ ]:
# take quite long, run it only when necessary 

# data pre-processing, lowercase, stopwords removal and stemming
docs_lower = [[w.lower() for w in doc] for doc in docs]
docs_alpha = [[w for w in doc if re.search('^[a-z]+$', w)] for doc in docs_lower]

stop_list = stopwords.words('english')            
docs_stop = [[w for w in doc if w not in stop_list] for doc in docs_alpha]

stemmer = PorterStemmer()
docs_stem = [[stemmer.stem(w) for w in doc] for doc in docs_stop]

In [52]:
print(docs_stem[0][:100])

['project', 'gutenberg', 'hous', 'borderland', 'william', 'hope', 'hodgson', 'ebook', 'use', 'anyon', 'anywher', 'cost', 'almost', 'restrict', 'whatsoev', 'may', 'copi', 'give', 'away', 'use', 'term', 'project', 'gutenberg', 'licens', 'includ', 'ebook', 'onlin', 'www', 'gutenberg', 'org', 'titl', 'hous', 'borderland', 'author', 'william', 'hope', 'hodgson', 'releas', 'date', 'novemb', 'ebook', 'last', 'updat', 'januari', 'languag', 'english', 'charact', 'set', 'encod', 'ascii', 'start', 'project', 'gutenberg', 'ebook', 'hous', 'borderland', 'produc', 'suzann', 'shell', 'sjaani', 'pg', 'distribut', 'proofread', 'hous', 'borderland', 'william', 'hope', 'hodgson', 'manuscript', 'discov', 'messr', 'tonnison', 'berreggnog', 'ruin', 'lie', 'south', 'villag', 'kraighten', 'west', 'ireland', 'set', 'father', 'whose', 'feet', 'tread', 'lost', 'aeon', 'open', 'door', 'listen', 'wind', 'muffl', 'roar', 'glisten', 'tear', 'round', 'moon', 'fanci', 'tread', 'vanish']


In [54]:
# Number of *unique* words in the books after data pre-processing
unique_words = list()
for book in docs:
    fdist_doc = nltk.FreqDist(book)
    filtered_word_freq = dict((word, freq) for word, freq in fdist_doc.items() if not word.isdigit())
    unique_words.append(len(filtered_word_freq))
print(unique_words)

[5954, 4836, 7045, 3564, 13958, 6965, 3940, 9260, 13660, 14241, 3367, 8521, 8574, 6777, 5845, 12639, 12480, 12167, 6664, 5873, 5735, 11245, 2940, 11407, 6270, 8297, 11186, 4423, 16388, 5731, 12850, 7725, 7315, 7030, 9402, 5513, 12880, 17822, 5240, 12854, 15200, 3633, 2678, 19801, 4877, 13496, 6457, 12304, 1926, 7095, 7280, 7262, 7079, 2714, 4650, 3018, 10638, 6465, 11658, 2008, 4635, 12481, 4038, 12773, 4876, 6035, 9234, 3786, 12197, 3286, 5089, 3769, 17808, 8257, 11543, 3553, 6945, 5222, 8033, 10611, 6824, 7831, 6781, 9189, 9190, 10615, 6826, 9246, 12028, 15656, 10747, 7375, 3477, 6450, 6863, 9291, 7804, 9160, 10454, 10415, 10404, 10500, 4705, 4709, 7817, 7814, 7809, 7807, 7809, 7803, 7809, 7880, 7857, 3943, 5012, 6073, 8237, 5811, 12154, 12154, 12219, 12259, 7744, 3266, 10801, 15164, 14211, 14167, 4915, 1888, 1890, 539, 7147, 7251, 7288, 5784, 3631, 2898, 7255, 7, 7072, 6890, 6889, 5277, 5208, 17791, 17791, 17870, 12020, 12067, 12075, 3684, 11078, 10888, 48]


In [56]:
dictionary = corpora.Dictionary(docs_stem)
# print(dictionary)
token_to_id = dictionary.token2id
print(type(token_to_id))
# print(token_to_id)

<class 'dict'>
